<a href="https://colab.research.google.com/github/HausReport/ClubRaiders/blob/master/notebooks/Squadrons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import sys

!{sys.executable} -m pip install --force-reinstall --upgrade "git+https://github.com/HausReport/ClubRaiders.git#egg=craid"
import craid.eddb.loader.DataProducer as dp
import pandas as pd
import logging
import plotly.graph_objs as go
from numpy import *


logging.getLogger().addHandler(logging.StreamHandler())
logging.getLogger().level = logging.DEBUG

arrays = dp.getDataArrays()
df: pd.DataFrame = arrays['dataFrame']

  Cloning https://github.com/HausReport/ClubRaiders.git to /tmp/pip-install-u96ztosh/craid
  Running command git clone -q https://github.com/HausReport/ClubRaiders.git /tmp/pip-install-u96ztosh/craid
Processing /root/.cache/pip/wheels/a2/06/bc/5c2246621220be254683eb2689a25dcb811762e2db0de8cf66/dash-1.13.4-cp36-none-any.whl
Processing /root/.cache/pip/wheels/35/b3/67/ca79d69f2b33930e0eef9c7197daafd3bb8e7c2133785dbcde/dash_core_components-1.10.1-cp36-none-any.whl
Processing /root/.cache/pip/wheels/22/c0/79/ef4c2b20a23dc8457ae75c3da5f63163db213d86e13e00fe26/dash_html_components-1.0.3-cp36-none-any.whl
Processing /root/.cache/pip/wheels/c4/17/bf/268d224583f366164cd1cf70d5fcf41ad7d584002ffd24537e/dash_table-4.8.1-cp36-none-any.whl
Processing /root/.cache/pip/wheels/76/04/53/2bc1654437aa5ff4e4da1bb40c3e9ebf1168282a2287d516d7/dash_renderer-1.5.1-cp36-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f2/28/2a03252dfb9ebf377f40fba6a7841b47083260bf8bd8e737b0c6952df83f/Flask-1.1

Loading from AWS
Loading from AWS
Found data file: /tmp/smol-factions.jsonl.gz
Found data file: /tmp/smol-factions.jsonl.gz
Read 663 lines of faction data
Read 663 lines of faction data
Found data file: /tmp/smol-systems_populated.jsonl.gz
Found data file: /tmp/smol-systems_populated.jsonl.gz
Read 172 lines of systems data
Read 172 lines of systems data


0:213,733,376
0.5:214,032,384
1:216,576,000

Populated club system keys
Populated club system keys
Club system lookup set has {172} items
Club system lookup set has {172} items
Populated faction instances: {1040} items.
Populated faction instances: {1040} items.
Found data file: /tmp/smol-stations.jsonl.gz
Found data file: /tmp/smol-stations.jsonl.gz
Read 734 lines of station data, adding 734

Read 734 lines of station data, adding 734




2:216,809,472


station controlling faction nexistQwent Research Base
station controlling faction nexistQwent Research Base
station controlling faction nexistQwent Research Base
station controlling faction nexistQwent Research Base
station controlling faction nexistQwent Research Base
station controlling faction nexistQwent Research Base
station controlling faction nexistQwent Research Base
station controlling faction nexistQwent Research Base
station controlling faction nexistQwent Research Base
station controlling faction nexistQwent Research Base
station controlling faction nexistQwent Research Base
station controlling faction nexistQwent Research Base
station controlling faction nexistQwent Research Base
station controlling faction nexistQwent Research Base
station controlling faction nexistQwent Research Base
station controlling faction nexistQwent Research Base
station controlling faction nexistQwent Research Base
station controlling faction nexistQwent Research Base
station controlling faction 

3:217,370,624


In [12]:
from craid.eddb.faction.SquadronXYZ import SquadronXYZ

In [23]:
sNames = []
sSystems = []
sRegions = []

for squad in SquadronXYZ.myDict.keys():
  systems = SquadronXYZ.myDict.get(squad)
  for sysName in systems.keys():
    sysLoc = systems.get(sysName)
    if sysLoc is None:
      continue
    sNames.append(squad)
    sSystems.append(sysName)
    sReg = RegionFactory.getRegion(sysLoc[0], sysLoc[1], sysLoc[2])
    sRegions.append(sReg.getNumber())

df = pd.DataFrame(list(zip(sNames, sSystems, sRegions)), 
               columns =['Squadron Name', 'System', 'Region']) 

In [24]:
df

,Squadron Name,System,Region
0,104th Phoenix Discovery Group,L 12-19,0
1,104th Phoenix Discovery Group,Woyo Mina,0
2,10th Fleet,Barnard's Star,4
3,10th Hawk Squadron,HIP 11728,2
4,10th Hawk Squadron,Mitnahas,2
...,...,...,...
16992,redback consortium,Wolf 1003,0
16993,space vans limited,60 Herculis,0
16994,space vans limited,Aguane,0
16995,space vans limited,Bany,0


In [58]:

def getSquadronsByRegion(region=1, cutoff=0) -> pd.DataFrame:
  tmp = df[ df['Region']==region].groupby(['Squadron Name']).agg( {'Region': 'count'}).reset_index()
  tmp.columns = ['Squadron Name','Systems in Region']
  tmp = tmp[ tmp['Systems in Region'] >= cutoff]
  return tmp.sort_values('Systems in Region', ascending=False)

In [59]:
getSquadronsByRegion(1,2)

,Squadron Name,Systems in Region
6,Operation Ida,18
11,The Hive,17
0,Anti Xeno Initiative,11
7,Sirius Inc,11
5,Hanks Alliance of Anti-Xeno,2


In [64]:
foo = getSquadronsByRegion(1,2)
print("| Squadron | # Systems |")
print("| -------- | --------- |")

for index, row in foo.iterrows():
    print("|" + row['Squadron Name'] + "|" + str(row['Systems in Region']) + "|")

| Squadron | # Systems |
| -------- | --------- |
|Operation Ida|18|
|The Hive|17|
|Anti Xeno Initiative|11|
|Sirius Inc|11|
|Hanks Alliance of Anti-Xeno|2|


In [1]:
for regNum in range(1,11):
  print(f"\n Region # {regNum}\n")
  if regNum==3:
    cutoff = 0
  else:
    cutoff = 2
  foo = getSquadronsByRegion(regNum,cutoff)
  print("| Squadron | # Systems |")
  print("| -------- | --------- |")

  for index, row in foo.iterrows():
    print("|" + row['Squadron Name'] + "|" + str(row['Systems in Region']) + "|")


 Region # 1



NameError: ignored